In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/polimi/ANN&DL

# Pre-operations

###Import libraries

In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image
from keras.applications.xception import preprocess_input

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

### Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Suppress warnings

In [ ]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

##Load data
Load the dataset to be used for classification

In [ ]:
dataset_dir_split = "dataset_challenge1"

In [ ]:
labels = []
for i in range(1,9):
  labels.append("Species"+str(i))

print(labels) 

In [ ]:
training_dir = os.path.join(dataset_dir_split, 'train') 
validation_dir = os.path.join(dataset_dir_split, 'val') 
test_dir = os.path.join(dataset_dir_split, 'test') 

In [ ]:
from keras.applications.efficientnet_v2 import preprocess_input

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug_generator= ImageDataGenerator(featurewise_center= False,
                                  samplewise_center= False,
                                  featurewise_std_normalization= True,
                                  samplewise_std_normalization= True,
                                  zca_whitening= False,
                                  rotation_range= 360,
                                  width_shift_range= 0.2,
                                  height_shift_range= 0.0,
                                  brightness_range=[0.2,0.7389633367497224],
                                  zoom_range= 0.2887325046461444,
                                  fill_mode= "nearest",
                                  horizontal_flip= False,
                                  vertical_flip= False)

In [ ]:
aug_train_gen = aug_generator.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None,
                                                       class_mode='categorical',
                                                       batch_size=8,
                                                       shuffle=True,
                                                       seed=seed)
                                             
                                             
valid_gen = ImageDataGenerator(
    ).flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=8,
                                               shuffle=False,
                                               seed=seed)
test_gen = ImageDataGenerator(
).flow_from_directory(directory=test_dir,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=8,
                                              shuffle=False,
                                              seed=seed)

###Weights

In [ ]:
from sklearn.utils import compute_class_weight

train_classes= aug_train_gen.classes
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_classes),
                                        y = train_classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_classes), class_weights))
class_weights

#Model
The model is an ensemble of two EfficientNet models, one EfficientNetS and one EfficientNetL, which works on the unbalanced dataset. Indeed, the dataset used for the training process is just the dataset of the competition splitted in train, validation and test. The fact that the dataset is unbalanced produces worst results especially in the less represented classes (e.g. Species 1). To solve this problem we used the compute_class_weight method offered by sklearn in order to weight the loss function. 

##*EfficientS*

The first model takes the CNN part from the EfficientNetV2S while the FC layer is characterized by a Dropout layer, a Dense layer, Leaky Relu, a Dropout layer and the output layer.
1. The following was the initial structure of the model where the hyperparameters, like, for instance, the number of units for the dense layer where chosen by us. 

In [ ]:
def build_model(hp):

  supernet = tfk.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3),
    pooling='avg',
    include_preprocessing=True
)
  
  elastic_lambda=1e-3

  # Use the supernet as feature extractor
  supernet.trainable = False

  inputs = tfk.Input(shape=(96,96,3))
  x = tfkl.Resizing(96,96, interpolation="bicubic")(inputs)
  x = supernet(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  x = tfkl.Dense( 
      units=512, 
      kernel_initializer = tfk.initializers.HeUniform(seed),
      kernel_regularizer=tf.keras.regularizers.L1L2(elastic_lambda,elastic_lambda))(x)
  x=tfkl.LeakyReLU()(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  outputs = tfkl.Dense(
      8, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


  # Connect input and output through the Model class
  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
  learning_rate = 1e-4
  tl_model.compile(
  optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),
  loss="categorical_crossentropy",
  metrics=["accuracy"],
  )
  return tl_model

2. To improve the results we decided to use hyperparameter tuning in order to find the best combination of hyperparameters for the model. This uses the keras-tuner module. We used the hyperparameter tuning also to find the best combination of augmentation.

#### install & import keras tuner

In [ ]:
!pip install keras-tuner -q

In [ ]:
import keras_tuner

####Model definition

In [ ]:
def build_model(hp):
  
################AUGMENTATION############################

  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  aug_train_gen = ImageDataGenerator(featurewise_center=hp.Boolean("featurewise_center"),
                                    samplewise_center=hp.Boolean("samplewise_center"),
                                    featurewise_std_normalization=hp.Boolean("featurewise_std_normalization"),
                                    samplewise_std_normalization=hp.Boolean("samplewise_std_normalization"),
                                    zca_whitening=hp.Boolean("zca_whitening"),
                                    rotation_range=hp.Int("rotation_range",min_value=0,max_value=360,step=10),
                                    width_shift_range=hp.Choice("width_shift_range",[0.0,0.2]),
                                    height_shift_range=hp.Choice("height_shift_range",[0.0,0.2]),
                                    brightness_range=[0.2,hp.Float("max_rotation",min_value=0.3,max_value=1.4)],
                                    zoom_range=hp.Float("zoom_range",min_value=0,max_value=1),
                                    fill_mode=hp.Choice("fill_mode",['nearest','constant','wrap','reflect']),
                                    horizontal_flip=hp.Boolean("horizontal_flip"),
                                    vertical_flip=hp.Boolean("vertical_flip"),
                                              ).flow_from_directory(directory=training_dir,
                                                        target_size=(96,96),
                                                        color_mode='rgb',
                                                        classes=None,
                                                        class_mode='categorical',
                                                        batch_size=8,
                                                        shuffle=True,
                                                        seed=seed)
                                              
                                              
  valid_gen = ImageDataGenerator(
      ).flow_from_directory(directory=validation_dir,
                                                target_size=(96,96),
                                                color_mode='rgb',
                                                classes=None, # can be set to labels
                                                class_mode='categorical',
                                                batch_size=8,
                                                shuffle=False,
                                                seed=seed)
  test_gen = ImageDataGenerator(
  ).flow_from_directory(directory=test_dir,
                                                target_size=(96,96),
                                                color_mode='rgb',
                                                classes=None, # can be set to labels
                                                class_mode='categorical',
                                                batch_size=8,
                                                shuffle=False,
                                                seed=seed)
##########################################################
  supernet = tfk.applications.EfficientNetV2S(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3),
    pooling='avg',
    include_preprocessing=True
)
  
  elastic_lambda= hp.Float("reg", min_value=1e-5, max_value=1e-2, sampling="log")

  # Use the supernet as feature extractor
  supernet.trainable = False

  inputs = tfk.Input(shape=(96,96,3))
  x = tfkl.Resizing(96,96, interpolation="bicubic")(inputs)
  x = supernet(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  x = tfkl.Dense( 
      units=hp.Int("units", min_value=32, max_value=512, step=32), 
      kernel_initializer = tfk.initializers.HeUniform(seed),
      kernel_regularizer=tf.keras.regularizers.L1L2(elastic_lambda,elastic_lambda))(x)
  x=tfkl.LeakyReLU()(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  outputs = tfkl.Dense(
      8, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


  # Connect input and output through the Model class
  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  tl_model.compile(
  optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),
  loss="categorical_crossentropy",
  metrics=["accuracy"],
  )
  return tl_model

#### Hyperparameter search
1. build the model

In [ ]:
build_model(keras_tuner.HyperParameters())

2. initialize the tuner. We decided to use a random search.

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory="my_dir",
    project_name="helloworld",
)

3. Run the search and get the results

In [ ]:
tuner.search(aug_train_gen,epochs=2, validation_data=valid_gen)
tuner.results_summary()

4. Get the best model found by the hyperparameter tuning

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(96,96,3))
best_model.summary()


####Transfer learning
We run a transfer learning step directly on the best model found by the hyperparameter tuning

In [ ]:
# Train the model
best_history = best_model.fit(
    aug_train_gen,
    epochs = 200,  
    validation_data = valid_gen,
    class_weight=class_weights,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(best_history['loss'], label='Training', alpha=.3, color='blue')
plt.plot(best_history['val_loss'], label='Validation', alpha=.8, color='red')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(best_history['accuracy'], label='Training', alpha=.8, color='blue')
plt.plot(best_history['val_accuracy'], label='Validation', alpha=.8, color='red')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Predict the test set with the CNN
predictions = best_model.predict(test_gen)
predictions.shape

pred=np.argmax(predictions,axis=1)
# Compute the confusion matrix
cm = confusion_matrix(test_gen.classes, pred)

pred=np.argmax(predictions,axis=-1)

# Compute the classification metrics
accuracy = accuracy_score(test_gen.classes, pred)
precision = precision_score(test_gen.classes, pred, average='macro')
recall = recall_score(test_gen.classes, pred, average='macro')
f1 = f1_score(test_gen.classes, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
best_model.save("ANN&DL_Model/TL_EFFNETS")

#### Fine tuning
Reload the model and run a step of fine tuning over it.

In [ ]:
# Re-load the model after transfer learning
path='ANN&DL_Model'
ft_model = tfk.models.load_model(os.path.join(path, 'TL_EFFNETS'))
ft_model.summary()

 Set all effnet layers to True. We kept the batch normalization layers freezed. We followed the keras documentation for Fine Tuning on the Efficient net. "The BatchNormalization layers need to be kept frozen. If they are also turned to trainable, the first epoch after unfreezing will significantly reduce accuracy."

In [ ]:

for i, layer in enumerate(ft_model.layers):  
   if not isinstance(layer, tfkl.BatchNormalization):
        layer.trainable=True
        
for i, layer in enumerate(ft_model.layers):  
    print(i, layer.name, layer.trainable)
ft_model.summary()

Freeze first 250 layers. The number of layers to freeze as been found experimentally. We tried with 150, 350 layers. The chosen combination was the one giving the best performance.

In [ ]:

for i, layer in enumerate(ft_model.get_layer('efficientnetv2-s').layers[:250]):  
        layer.trainable=False
        
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-s').layers):  
     if isinstance(layer, tfkl.BatchNormalization):
        layer.trainable=False
        
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-s').layers):  
   print(i, layer.name, layer.trainable)
ft_model.summary()

For the finetuning we used a learning rate 10 times smaller than the one used for the transfer learning phase.

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 256,
    epochs = 200,
    validation_data = valid_gen,
    class_weight=w,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Predict the test set with the CNN
predictions = ft_model.predict(test_gen)
predictions.shape

pred=np.argmax(predictions,axis=1)
# Compute the confusion matrix
cm = confusion_matrix(test_gen.classes, pred)

pred=np.argmax(predictions,axis=-1)

# Compute the classification metrics
accuracy = accuracy_score(test_gen.classes, pred)
precision = precision_score(test_gen.classes, pred, average='macro')
recall = recall_score(test_gen.classes, pred, average='macro')
f1 = f1_score(test_gen.classes, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
ft_model.save('ANN&DL_Model/FT_EFFNETS_85')

##*EfficientL*

The second model takes the CNN part from the EfficientNetV2L while the FC layer is characterized by a Dropout layer, a Dense layer, Leaky Relu, a Dropout layer and the output layer.
1. The following was the initial structure of the model where the hyperparameters, like, for instance, the number of units for the dense layer where chosen by us. 

In [ ]:
def build_model(hp):

  supernet = tfk.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3),
    pooling='avg',
    include_preprocessing=True
)
  
  elastic_lambda=1e-3

  # Use the supernet as feature extractor
  supernet.trainable = False

  inputs = tfk.Input(shape=(96,96,3))
  x = tfkl.Resizing(96,96, interpolation="bicubic")(inputs)
  x = supernet(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  x = tfkl.Dense( 
      units=512, 
      kernel_initializer = tfk.initializers.HeUniform(seed),
      kernel_regularizer=tf.keras.regularizers.L1L2(elastic_lambda,elastic_lambda))(x)
  x=tfkl.LeakyReLU()(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  outputs = tfkl.Dense(
      8, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


  # Connect input and output through the Model class
  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
  learning_rate = 1e-4
  tl_model.compile(
  optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),
  loss="categorical_crossentropy",
  metrics=["accuracy"],
  )
  return tl_model

2. To improve the results we decided to use hyperparameter tuning in order to find the best combination of hyperparameters for the model. This uses the keras-tuner module. We used the hyperparameter tuning also to find the best combination of augmentation.

#### install & import keras tuner

In [ ]:
!pip install keras-tuner -q

In [ ]:
import keras_tuner

####Model definition

In [ ]:
def build_model(hp):
  
################AUGMENTATION############################

  from tensorflow.keras.preprocessing.image import ImageDataGenerator
  aug_train_gen = ImageDataGenerator(featurewise_center=hp.Boolean("featurewise_center"),
                                    samplewise_center=hp.Boolean("samplewise_center"),
                                    featurewise_std_normalization=hp.Boolean("featurewise_std_normalization"),
                                    samplewise_std_normalization=hp.Boolean("samplewise_std_normalization"),
                                    zca_whitening=hp.Boolean("zca_whitening"),
                                    rotation_range=hp.Int("rotation_range",min_value=0,max_value=360,step=10),
                                    width_shift_range=hp.Choice("width_shift_range",[0.0,0.2]),
                                    height_shift_range=hp.Choice("height_shift_range",[0.0,0.2]),
                                    brightness_range=[0.2,hp.Float("max_rotation",min_value=0.3,max_value=1.4)],
                                    zoom_range=hp.Float("zoom_range",min_value=0,max_value=1),
                                    fill_mode=hp.Choice("fill_mode",['nearest','constant','wrap','reflect']),
                                    horizontal_flip=hp.Boolean("horizontal_flip"),
                                    vertical_flip=hp.Boolean("vertical_flip"),
                                              ).flow_from_directory(directory=training_dir,
                                                        target_size=(96,96),
                                                        color_mode='rgb',
                                                        classes=None,
                                                        class_mode='categorical',
                                                        batch_size=8,
                                                        shuffle=True,
                                                        seed=seed)
                                              
                                              
  valid_gen = ImageDataGenerator(
      ).flow_from_directory(directory=validation_dir,
                                                target_size=(96,96),
                                                color_mode='rgb',
                                                classes=None, # can be set to labels
                                                class_mode='categorical',
                                                batch_size=8,
                                                shuffle=False,
                                                seed=seed)
  test_gen = ImageDataGenerator(
  ).flow_from_directory(directory=test_dir,
                                                target_size=(96,96),
                                                color_mode='rgb',
                                                classes=None, # can be set to labels
                                                class_mode='categorical',
                                                batch_size=8,
                                                shuffle=False,
                                                seed=seed)
##########################################################
  supernet = tfk.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_shape=(96,96,3),
    pooling='avg',
    include_preprocessing=True
)
  
  elastic_lambda= hp.Float("reg", min_value=1e-5, max_value=1e-2, sampling="log")

  # Use the supernet as feature extractor
  supernet.trainable = False

  inputs = tfk.Input(shape=(96,96,3))
  x = tfkl.Resizing(96,96, interpolation="bicubic")(inputs)
  x = supernet(x) 
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  x = tfkl.Dense( 
      units=hp.Int("units", min_value=32, max_value=512, step=32), 
      kernel_initializer = tfk.initializers.HeUniform(seed),
      kernel_regularizer=tf.keras.regularizers.L1L2(elastic_lambda,elastic_lambda))(x)
  x=tfkl.LeakyReLU()(x)
  x = tfkl.Dropout(0.5, seed=seed)(x)    #0.3
  outputs = tfkl.Dense(
      8, 
      activation='softmax',
      kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)


  # Connect input and output through the Model class
  tl_model = tfk.Model(inputs=inputs, outputs=outputs, name='model')
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  tl_model.compile(
  optimizer=tfk.optimizers.Adam(learning_rate=learning_rate),
  loss="categorical_crossentropy",
  metrics=["accuracy"],
  )
  return tl_model

#### Hyperparameter search
1. build the model

In [ ]:
build_model(keras_tuner.HyperParameters())

2. initialize the tuner. We decided to use a random search.

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    overwrite=False,
    directory="my_dir",
    project_name="tuningL",
)

3. Run the search and get the results

In [ ]:
tuner.search(aug_train_gen,epochs=2, validation_data=valid_gen)
tuner.results_summary()

4. Get the best model found by the hyperparameter tuning

In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build(input_shape=(96,96,3))
best_model.summary()


####Transfer learning
We run a transfer learning step directly on the best model found by the hyperparameter tuning

In [ ]:
# Train the model
best_history = best_model.fit(
    aug_train_gen,
    epochs = 200,  
    validation_data = valid_gen,
    class_weight=class_weights,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(best_history['loss'], label='Training', alpha=.3, color='blue')
plt.plot(best_history['val_loss'], label='Validation', alpha=.8, color='red')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(best_history['accuracy'], label='Training', alpha=.8, color='blue')
plt.plot(best_history['val_accuracy'], label='Validation', alpha=.8, color='red')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Predict the test set with the CNN
predictions = best_model.predict(test_gen)
predictions.shape

pred=np.argmax(predictions,axis=1)
# Compute the confusion matrix
cm = confusion_matrix(test_gen.classes, pred)

pred=np.argmax(predictions,axis=-1)

# Compute the classification metrics
accuracy = accuracy_score(test_gen.classes, pred)
precision = precision_score(test_gen.classes, pred, average='macro')
recall = recall_score(test_gen.classes, pred, average='macro')
f1 = f1_score(test_gen.classes, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
best_model.save("ANN&DL_Model/TL_EFFNETL")

#### Fine tuning
Reload the model and run a step of fine tuning over it.

In [ ]:
# Re-load the model after transfer learning
path='ANN&DL_Model'
ft_model = tfk.models.load_model(os.path.join(path, 'TL_EFFNETL'))
ft_model.summary()

 Set all effnet layers to True. We kept the batch normalization layers freezed. We followed the keras documentation for Fine Tuning on the Efficient net. "The BatchNormalization layers need to be kept frozen. If they are also turned to trainable, the first epoch after unfreezing will significantly reduce accuracy."

In [ ]:

for i, layer in enumerate(ft_model.layers):  
   if not isinstance(layer, tfkl.BatchNormalization):
        layer.trainable=True
        
for i, layer in enumerate(ft_model.layers):  
    print(i, layer.name, layer.trainable)
ft_model.summary()

Freeze first 250 layers. The number of layers to freeze as been found experimentally. We tried with 150, 350 layers. The chosen combination was the one giving the best performance.

In [ ]:

for i, layer in enumerate(ft_model.get_layer('efficientnetv2-l').layers[:650]):  
        layer.trainable=False
        
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-l').layers):  
     if isinstance(layer, tfkl.BatchNormalization):
        layer.trainable=False
        
for i, layer in enumerate(ft_model.get_layer('efficientnetv2-l').layers):  
   print(i, layer.name, layer.trainable)
ft_model.summary()

For the finetuning we used a learning rate 10 times smaller than the one used for the transfer learning phase.

In [ ]:
# Compile the model
ft_model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-5), metrics='accuracy')

In [ ]:
# Fine-tune the model
ft_history = ft_model.fit(
    x = aug_train_gen,
    batch_size = 256,
    epochs = 200,
    validation_data = valid_gen,
    class_weight=w,
    callbacks = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=10, restore_best_weights=True)]
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(tl_history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_loss'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['loss'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_loss'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Categorical Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(tl_history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(tl_history['val_accuracy'], label='Transfer Learning', alpha=.8, color='#4D61E2')
plt.plot(ft_history['accuracy'], alpha=.3, color='#2ABC3D', linestyle='--')
plt.plot(ft_history['val_accuracy'], label='Fine Tuning', alpha=.8, color='#2ABC3D')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
# Predict the test set with the CNN
predictions = ft_model.predict(test_gen)
predictions.shape

pred=np.argmax(predictions,axis=1)
# Compute the confusion matrix
cm = confusion_matrix(test_gen.classes, pred)

pred=np.argmax(predictions,axis=-1)

# Compute the classification metrics
accuracy = accuracy_score(test_gen.classes, pred)
precision = precision_score(test_gen.classes, pred, average='macro')
recall = recall_score(test_gen.classes, pred, average='macro')
f1 = f1_score(test_gen.classes, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
ft_model.save('ANN&DL_Model/FT_EFFNETL')

##Ensemble
Now having the two models we build the ensemble of the two.
The ensemble method puts together several models and the output of the ensemble model will be the average of the outputs of the different input models.

In [ ]:
def ensemble(models, model_input):
    
    model_outputs=[]
    for i in range(len(models)):
      model=models[i]
      model._name="model" + str(i)
      model_outputs.append(model(model_input))
    
    ensemble_output = tfkl.Average()(model_outputs)
    ensemble_model = tf.keras.Model(inputs=model_input, outputs=ensemble_output)
    
    return ensemble_model

In [ ]:
effnetS=tfk.models.load_model(os.path.join(path, 'FT_EFFNETS'))
effnetL=tfk.models.load_model(os.path.join(path, 'FT_EFFNETL'))
inputs = tfk.Input(shape=(96,96,3))
final_model=ensemble([effnetS,effnetL],inputs)

In [ ]:
# Predict the test set with the CNN
predictions = final_model.predict(test_gen)
predictions.shape

pred=np.argmax(predictions,axis=1)
# Compute the confusion matrix
cm = confusion_matrix(test_gen.classes, pred)

pred=np.argmax(predictions,axis=-1)

# Compute the classification metrics
accuracy = accuracy_score(test_gen.classes, pred)
precision = precision_score(test_gen.classes, pred, average='macro')
recall = recall_score(test_gen.classes, pred, average='macro')
f1 = f1_score(test_gen.classes, pred, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

# Plot the confusion matrix
plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=labels, yticklabels=labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()

In [ ]:
final_model.save('ANN&DL_Model/Final_model')